In [11]:
import ase
import ase.io
from ase.io import xsf
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [12]:
dataframe_df2 = pd.read_pickle("DataFrame_df2.pkl")

In [13]:
dataframe_df2.head()

,uuid,structure,cell,volume,relevant_dimension_of_highest_dimensionality_substructure,binding_energy_per_substructure,binding_energy,label,number_of_atoms_in_bulk,number_of_highest_dimensionality_substructures_in_bulk,sub_structure_spg,sub_structure_dim,energy_bulk,sub_structure_formula,PMG Structure from ASE Atoms
0,dec2d070-d2ad-41f3-b742-786368ade33a,"(Atom('Cd', [-0.0, -0.0, 29.0441583], index=0)...","[[2.1044124705, 1.2149831063, 34.279850741], [...",525.885600,15.340954,-0.165000,10.755509,EE,15,5,"[C2/m (12), C2/m (12), C2/m (12), C2/m (12), C...","[2, 2, 2, 2, 2]",-17198.871453,"[CdI2, CdI2, CdI2, CdI2, CdI2]","[[ 0. 0. 29.0441583] Cd, [ 0. ..."
1,51e630e8-d94f-421c-bd1e-3e2f298e740c,"(Atom('Fe', [0.0, 0.0, 0.0], index=0), Atom('C...","[[3.35814166734363, 3.21879556184565e-14, -3.2...",52.188194,9.766268,-0.149888,15.347484,EE,3,1,[P-3m1 (164)],[2],-5361.474444,[Cl2Fe],"[[0. 0. 0.] Fe, [0. 2.06980071 4.36433..."
2,527e7b32-d211-4743-9631-120122819190,"(Atom('Er', [2.1386, 4.301805, 3.209025], inde...","[[2.14789490163679, -12.5878501270869, 0.0], [...",232.520056,18.471785,-0.454048,24.580650,EE,8,1,[P2_1/m (11)],[2],-26090.090963,[Er2Te6],"[[2.1386 4.301805 3.209025] Er, [ 2.1386 -..."
3,37bf14b1-6529-436c-a519-e1ad9b208784,"(Atom('Ba', [0.0, 0.0, 9.5697], index=0), Atom...","[[-2.0745739339, 2.0745739339, 7.4278571834], ...",127.873741,17.215428,-0.747937,43.445746,PE,3,1,[C2/m (12)],[2],-2866.173187,[Ba2Hg],"[[0. 0. 9.5697] Ba, [0. 0. 5.6..."
4,e7a9cec3-f11a-4d16-a8c5-ebd758bb6592,"(Atom('Ba', [0.0, 2.414, 1.49473], index=0), A...","[[4.6829418165, 0.0, 0.0], [0.0, 4.6829418165,...",166.823529,21.929944,-0.532853,24.297979,EE,6,1,[P4/nmm (129)],[2],-3217.539658,[Ba2H2I2],"[[0. 2.414 1.49473] Ba, [2.414 0. ..."


***We are going to use the crystal graph convolutional neural network: https://github.com/txie-93/cgcnn***

*The code is compiled here: pc37:/home/mvahdat/Documents/Crystal-Graph-Convolutional-Neural-Networks/Cloned-Repo/cgcnn*

*Remember to activate the environment: conda activate cgcnn*

***CIF files recording the labeled structures:***

In [14]:
i = 0
for structure in list(dataframe_df2['structure']): 
    ase.io.write('./CIFs/{}.cif'.format(i), structure, format = 'cif')
    i += 1

In [15]:
df_binary = dataframe_df2.copy()
df_binary['label'] = df_binary['label'].replace(to_replace='PE', value = 'EE')
df_binary['label'] = df_binary['label'].replace(to_replace='EE', value = 1)
df_binary['label'] = df_binary['label'].replace(to_replace='HBE', value = 0)

***id_prop.csv: a CSV file with two columns. The first columns recordes a unique ID for each crystal, and the second column recordes the value of target propetry:***

In [16]:
df_binary['label'].to_csv('id_prop.csv')

***Transfer CIF files and id_prop.csv files to pc37 and run the code:***

In [21]:
cgcnn_binary_result = pd.read_csv('cgcnn_binary_result.csv', header=None)
cgcnn_binary_result.head()

,0,1,2
0,2381,0.0,0.003430
1,587,1.0,0.874333
2,1001,1.0,0.523773
3,1367,1.0,0.861641
4,2053,0.0,0.214794


In [22]:
y = result[1].values
predictions = [round(i) for i in list(result[2])] ### Round the values obtained between 0 to 1

***Evaluation:***

In [23]:
from sklearn.metrics import classification_report, confusion_matrix

In [24]:
print(classification_report(y,predictions))
print('\n')
print(confusion_matrix(y,predictions))

              precision    recall  f1-score   support

         0.0       0.65      0.62      0.63       112
         1.0       0.88      0.90      0.89       366

    accuracy                           0.83       478
   macro avg       0.77      0.76      0.76       478
weighted avg       0.83      0.83      0.83       478



[[ 69  43]
 [ 37 329]]
